In [1]:
import numpy as np
import arepo
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib as mpl
import h5py as h5

import astropy.units as u

from joblib import Parallel, delayed

In [2]:
import illustris_python as il
TNGbase = '/n/holylfs05/LABS/hernquist_lab/IllustrisTNG/Runs/L35n2160TNG/output/'

In [3]:
def get_time(time, redshift=False, 
             Omega0=0.3089, 
             OmegaLambda=0.6911,
             HubbleParam=0.6774):
    HUBBLE = 3.2407789e-18
    SEC_PER_MEGAYEAR = 3.15576e13
    
    if redshift:
        a = 1./(1.+time)
    else:
        a = time
    
    fac = 2. / (3. * np.sqrt(OmegaLambda))
    ans = fac * np.arcsinh(np.sqrt(a**3 * OmegaLambda/Omega0))

    ans /= HUBBLE * HubbleParam
    ans /= SEC_PER_MEGAYEAR * 1000
    
    return ans

In [4]:
meta = {}
meta['snap_list'] = np.arange(100)
meta['header'] = []
meta['redshift'] = []
meta['scale_factor'] = []
meta['time'] = []
meta['time_lookback'] = []

t0 = get_time(1.)

for i in meta['snap_list']:
    header = arepo.Snapshot(TNGbase, i, onlyHeader=True)
    
    meta['header'].append(header)
    meta['redshift'].append(header.Redshift)
    meta['scale_factor'].append(header.Time)
    meta['time'].append(get_time(header.Time))
    meta['time_lookback'].append(t0 - get_time(header.Time))

meta['redshift'] = np.array(meta['redshift'])
meta['scale_factor'] = np.array(meta['scale_factor'])
meta['time'] = np.array(meta['time'])
meta['time_lookback'] = np.array(meta['time_lookback'])

In [6]:
snapnum = 99
z = meta['redshift'][snapnum]
tuniv = meta['time'][snapnum]

print('snapnum=', snapnum)
print('z=', z)
print('tuniv=', tuniv)

snapnum= 99
z= 2.220446049250313e-16
tuniv= 13.802719985485238


In [8]:
subID = 392276
tree = il.sublink.loadTree(TNGbase, snapnum, subID)

In [22]:
def maxPastMass(tree, index, ptNum=4):
    """ Get maximum past mass (of the given partType) along the main branch of a subhalo
        specified by index within this tree. """

    branchSize = tree['MainLeafProgenitorID'][index] - tree['SubhaloID'][index] + 1
    masses = tree['SubhaloMassType'][index: index + branchSize, ptNum]
    return np.max(masses)

def listMergers(tree, minMassRatio=1e-10, massPartNum=4, index=0, alongFullTree=False):
    """ Calculate the number of mergers, along the main progenitor branch, in this sub-tree 
    (optionally above some mass ratio threshold). If alongFullTree, count across the full 
    sub-tree and not only along the MPB. """
    # verify the input sub-tree has the required fields
    reqFields = ['SubhaloID', 'NextProgenitorID', 'MainLeafProgenitorID',
                 'FirstProgenitorID', 'SubhaloMassType']

    if not set(reqFields).issubset(tree.keys()):
        raise Exception('Error: Input tree needs to have loaded fields: '+', '.join(reqFields))

    num = 0
    invMassRatio = 1.0 / minMassRatio

    # walk back main progenitor branch
    rootID = tree['SubhaloID'][index]
    fpID   = tree['FirstProgenitorID'][index]

    while fpID != -1:
        fpIndex = index + (fpID - rootID)
        fpMass  = maxPastMass(tree, fpIndex, ptNum=massPartNum)

        # explore breadth
        npID = tree['NextProgenitorID'][fpIndex]

        while npID != -1:
            npIndex = index + (npID - rootID)
            npMass  = maxPastMass(tree, npIndex, ptNum=massPartNum)

            # count if both masses are non-zero, and ratio exceeds threshold
            if fpMass > 0.0 and npMass > 0.0:
                ratio = npMass / fpMass

                if ratio >= minMassRatio and ratio <= invMassRatio:
                    print('npidx, z = ', tree['SnapNum'][npIndex], round(meta['redshift'][tree['SnapNum'][npIndex]], 3))
                    print('fpidx, z = ', tree['SnapNum'][fpIndex], round(meta['redshift'][tree['SnapNum'][fpIndex]], 3))
                    print('mass ratio = ', round(1/ratio, 3))
                    print()
                    num += 1

            npID = tree['NextProgenitorID'][npIndex]

            # count along full tree instead of just along the MPB? (non-standard)
            if alongFullTree:
                if tree['FirstProgenitorID'][npIndex] != -1:
                    numSubtree = numMergers(tree, minMassRatio=minMassRatio, massPartType=massPartType, index=npIndex)
                    num += numSubtree

        fpID = tree['FirstProgenitorID'][fpIndex]

    return num

In [23]:
listMergers(tree, 0.1, massPartNum=4)

npidx, z =  11 7.005
fpidx, z =  12 6.492
mass ratio =  3.906

npidx, z =  8 8.012
fpidx, z =  9 7.595
mass ratio =  6.371

npidx, z =  1 14.989
fpidx, z =  1 14.989
mass ratio =  8.75

npidx, z =  1 14.989
fpidx, z =  1 14.989
mass ratio =  7.203



4

In [41]:
# def get_maxpastmass_sat(snapnum, subID):
snapnum = 99
subID = 392276
if True:
    sub = il.groupcat.loadSingle(TNGbase, snapnum, subhaloID=subID)
    grpID = sub['SubhaloGrNr']
    grp = il.groupcat.loadSingle(TNGbase, snapnum, haloID=grpID)
    
    subIDs_ofgrp = np.arange(grp['GroupFirstSub'], grp['GroupFirstSub']+grp['GroupNsubs'])
    maxpastmass = np.zeros_like(subIDs_ofgrp, dtype=float)
    
    for i,subID_ in enumerate(tqdm(subIDs_ofgrp)):
        tree_ = il.sublink.loadTree(TNGbase, snapnum, subID_, onlyMPB=True)
        maxpastmass[i] = np.max(tree_['SubhaloMassType'][:,4])

100%|██████████| 2345/2345 [01:02<00:00, 37.53it/s]


In [47]:
maxmass_sorted = np.flip(np.sort(maxpastmass))
print((maxmass_sorted/maxmass_sorted[0])[:10])

[1.         0.57663204 0.37673053 0.07996716 0.04669543 0.01284725
 0.01234276 0.01055629 0.00593234 0.00526202]
